In [1]:
import dotenv
dotenv.load_dotenv('.env', override=True)

False

In [2]:
import os
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
            os.environ['NEO4J_URI'], 
            auth=(os.environ['NEO4J_USERNAME'], os.environ['NEO4J_PASSWORD'],))

In [3]:
import csv
from tqdm import tqdm

def load_articles(csv_file, start_idx=0, end_idx=None):
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=';')
        with driver.session() as session:
            for i, row in enumerate(tqdm(reader, desc="Loading Articles", unit="article")):
                if i < start_idx:
                    continue

                if end_idx is not None and i >= end_idx:
                    break
                
                session.write_transaction(create_article, row)

def create_article(tx, row):
    title = row['title'].strip()
    abstract = row['abstract'].strip()
    date_publication = row['date_publication']
    keywords = row['keywords'].split(',')

    # Merge the article node
    tx.run("""
    CREATE (a:Article {name: $title})
    SET a.abstract = $abstract, a.date_publication = $date_publication
    WITH a
    UNWIND $keywords AS keyword
    MERGE (b:Keyword {text: keyword})
    MERGE (b)<-[:CONTAIN]-(a)
    """, title=title, abstract=abstract, date_publication=date_publication, keywords=keywords)

In [64]:
load_articles('./data.csv', start_idx=0, end_idx=50)

Loading Articles: 0article [00:00, ?article/s]C:\Users\Lenovo E585 (9-3)\AppData\Local\Temp\ipykernel_8212\2343436497.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_article, row)
Loading Articles: 50article [00:06,  8.22article/s]


In [8]:
%cd ..

c:\Auto\RAG-Mistral


In [15]:
import pandas as pd 

df = pd.read_csv('./scopus_data_cleaned.csv', sep=';')
df.head(10)

,title,abstract,date_publication,keywords
0,From Misaligned Sub-Saturns to Aligned Brown D...,We present a pattern emerging from stellar obl...,2024-12-05,"['exoplanet dynamics (490)', ' star-planet int..."
1,Uniting the Observed Dynamical Dark Energy Pre...,"Recent results from Type Ia Supernovae (SNe), ...",2024-12-05,"['supernovae', 'physics', 'lambdaroman_λcdm', ..."
2,Journey to the center of the common envelope e...,Three-dimensional hydrodynamical simulations o...,2024-12-05,['Key Words.: Hydrodynamics – binaries: close ...
3,Baryon Acoustic Oscillations from galaxy surve...,We conducted a review of the fundamental aspec...,2024-12-05,"['galaxies', 'alphaitalic_α', 'spectroscopic',..."
4,Optical constraints on the coldest metal-poor ...,Context. The coldest metal-poor population mad...,2024-12-05,"['coldest', 'old', 'dwarfs', 'archaeological',..."
5,The Red Supergiant Problem: As Seen from the L...,"The red supergiant (RSG) problem, which descri...",2024-12-05,['Supernova Progenitors']
6,The Thermal Sunyaev-Zel’dovich Effect from the...,The thermal Sunyaev-Zel’dovich (tSZ) effect ar...,2024-12-05,"['toℓsuperscript103superscript104', 'electrons..."
7,Investigating the role of nuclear parametersin...,Recent studies based on the relativistic mean ...,2024-12-05,"['physics', 'astrophysics', 'astrophysical', '..."
8,DARWEN: Data-driven Algorithm for Reduction of...,Context. Exoplanet atmospheric modeling is adv...,2024-12-05,['Key Words.: Astrochemistry –Methods: numeric...
9,Water Ice in the Edge-On Orion Silhouette Disk...,We examine images of the protoplanetary disk 1...,2024-12-05,"['orion', 'protoplanetary', 'nebula', 'midplan..."


In [29]:
list_keywords = df["keywords"].values
clean_keywords = []
for keywords in list_keywords:
    temp = keywords[1:len(keywords)-1]
    temp = temp.replace("'", "")
    temp = temp.replace(" ", "")
    clean_keywords.append(temp)

df["keywords"] = clean_keywords

In [31]:
df.sample(10)

,title,abstract,date_publication,keywords
559,Generation of a reflected wave in an inhomogen...,Abstract: In this work we present the results ...,2024-12-04,"electric,inhomogeneous,particlewith,waveguides..."
813,Non-linear Faraday Precession of Light Polariz...,Abstract: We investigate the propagation of el...,2024-12-04,"electromagnetic,radiation,boltzmann,equations,..."
557,Dielectric Loaded Waveguide Terahertz LINACs,Abstract: Dielectric loaded waveguides (DLWs) ...,2024-12-04,"thedielectric,accelerators,medicalapplications..."
664,Wavepacket and Reduced-Density Approaches for ...,Abstract: Excitation-energy transfer (EET) and...,2024-12-04,"laser,wavepackets,quantum,multiconfiguration,s..."
628,Real-time tracking the energy flow in cluster ...,Abstract: While photodissociation of molecular...,2024-12-02,"photoelectron,decaycharacteristic,nanodroplets..."
748,QuakeFormer: A Uniform Approach to Earthquake ...,Abstract: Ground motion prediction (GMP) model...,2024-12-01,"quakeformeroffers,quakeformer,seismicrecording..."
1011,Non-Symmetric Askey--Wilson Shift Operators,Abstract: We classify the shift operators for ...,2024-12-04,"polynomials,polynomialsand,polynomialsusing,de..."
863,Multi-Electrode Dielectric Barrier Discharge A...,Abstract: Dielectric barrier discharge (DBD) p...,2024-12-02,"electrohydrodynamic,electrodeconfiguration,pow..."
638,Social patch foraging theory in an egalitarian...,"Abstract: Foraging is a widespread behavior, a...",2024-12-03,"foraging,mechanistic,food,hypotheses,foranalyt..."
418,"Squashed 7-spheres, octonions and the swampland",Abstract: The entire eigenvalue spectrum of th...,2024-12-05,"supermultiplet,2024,eigenvalue,isometryreprese..."


In [33]:
df.to_csv('scopus_data_cleaned.csv', index=False, header=True, sep=';')

In [34]:
load_articles('./scopus_data_cleaned.csv', start_idx=0)

Loading Articles: 0article [00:00, ?article/s]C:\Users\Auto\AppData\Local\Temp\ipykernel_11020\2343436497.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_article, row)
Loading Articles: 195article [00:50,  3.69article/s][#E7A3]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('464aa553.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Transaction failed and will be retried in 1.1860029653492992s (Failed to read from defunct connection IPv4Address(('464aa553.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))))
[#E7A2]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.161.242', 7687)) (ResolvedIPv4Address(('34.126.161.242', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.9791282743678624s (Unable to retrieve routing informat